In [1]:
import sys, datetime
sys.path.insert(1, '../../scripts/')
from s3_support import *
%matplotlib inline

# building where & join

In [8]:
def get_sql_for_filters(filters):
    filters_sql = []
    joins_clauses = []
    
    for f in filters:
        if f.lower() == 'ntee':
            param = filters[f]
            filters_sql.append("o.ntee='{}'".format(param))
            joins_clauses.append("organization as o on t.org=o.id")
        elif f.lower() == 'tag':
            param = filters[f]
            filters_sql.append("o.tag contains '{}'".format(param))
            joins_clauses.append("organization as o on t.org=o.id")
        elif f.lower() == 'timeframe':
            if filters[f] == '1week':
                min_date = datetime.datetime.now() - datetime.timedelta(weeks=1)
            elif filters[f] == '1month':
                min_date = datetime.datetime.now() - datetime.timedelta(weeks=4)
            elif filters[f] == '3months':
                min_date = datetime.datetime.now() - datetime.timedelta(weeks=12)
            elif filters[f] == '6months':
                min_date = datetime.datetime.now() - datetime.timedelta(weeks=24)
            elif filters[f] == '1year':
                min_date = datetime.datetime.now() - datetime.timedelta(weeks=52)
            
            filters_sql.append("t.date>='{}'".format(min_date))
            
    return filters_sql, set(joins_clauses)

In [9]:
params = {'ntee': '32235', 'timeframe': '1year', 'tag': 'top20'}
get_sql_for_filters(params)

(["o.ntee='32235'",
  "t.date>='2021-08-25 15:12:13.274330'",
  "o.tag contains 'top20'"],
 {'organization as o on t.o=o.id'})

# build sql

In [16]:
def get_stat(select, where_statements, joins=None, group=None):
    q = select
    
    if joins is not None:
        q += ' left join ' + ' left join '.join(joins)
        
    q += ' where ' + " and ".join(where_statements)
    
    if group is not None:
        q += ' group by ' + ", ".join(group)
    
    #df = redshift_query_read(q, schema='production')
    return q

In [17]:
select = 'select sum(t.amount) as stat from transactions as t'
where, joins = get_sql_for_filters(params)

get_stat(select, where, joins=joins)

"select sum(t.amount) as stat from transactions as t left join organization as o on t.o=o.id where o.ntee='32235' and t.date>='2021-08-25 15:20:46.063712' and o.tag contains 'top20'"

# API testing

In [18]:
import requests, json

In [21]:
url = "http://52.2.34.90:8900/orginsights/"

In [22]:
rsp = requests.post(url)
rsp.text

'{\n  "error": "Error attempting to calculate insight; \'insights\'", \n  "success": 0\n}\n'

In [31]:
data = {
    'insights': 'contribution_count,contribution_total',
    'timeframe': '3months'
}
rsp = requests.post(url, data=data)
rsp.text

'{\n  "stats": [\n    {\n      "insight": "contribution_count", \n      "stat": "166"\n    }, \n    {\n      "insight": "contribution_total", \n      "stat": "19696.0392468211"\n    }\n  ], \n  "success": 1\n}\n'

In [32]:
data = {
    'insights': 'contribution_count,contribution_total,average_contribution_amount',
    'timeframe': '1year'
}
rsp = requests.post(url, data=data)
rsp.text

'{\n  "stats": [\n    {\n      "insight": "contribution_count", \n      "stat": "668"\n    }, \n    {\n      "insight": "contribution_total", \n      "stat": "83863.2639710561"\n    }, \n    {\n      "insight": "average_contribution_amount", \n      "stat": "125.427105657801"\n    }\n  ], \n  "success": 1\n}\n'

In [34]:
data = {
    'insights': 'average_contribution_amount',
    'timeframe': '1year',
    'segment': 'Youth'
}
rsp = requests.post(url, data=data)
rsp.text

'{\n  "stats": [\n    {\n      "insight": "average_contribution_amount", \n      "stat": "191.353604947876"\n    }\n  ], \n  "success": 1\n}\n'

In [35]:
data = {
    'insights': 'contribution_count,contribution_total,average_contribution_amount',
    'timeframe': '1year',
    'segment': 'Education'
}
rsp = requests.post(url, data=data)
rsp.text

'{\n  "stats": [\n    {\n      "insight": "contribution_count", \n      "stat": "425"\n    }, \n    {\n      "insight": "contribution_total", \n      "stat": "89942.7583514103"\n    }, \n    {\n      "insight": "average_contribution_amount", \n      "stat": "211.5814236873"\n    }\n  ], \n  "success": 1\n}\n'